In [1]:
import pandas as pd
from detoxify import Detoxify
from tqdm.auto import tqdm
from IPython.display import clear_output
import os

ModuleNotFoundError: No module named 'detoxify'

In [24]:
arquivoDeLeitura = "comments_NegoBorel.json"
arquivoDeSaida   = "comentarios_NegoBorel_Classificados.json"

In [25]:
# Lendo o arquivo JSON com os comentários
dfComentarios = pd.read_json(arquivoDeLeitura, encoding="utf-8")

In [26]:
print(dfComentarios.shape)
dfComentarios.head(5)

(144695, 5)


,id,author,message,publishedAt,likeCount
0,Ugw4HzjaXSliZRj9zPd4AaABAg,@danielesena7780,Musica muito boa,2025-05-07T00:10:29Z,0
1,UgyLFdfcGra2nxstkrt4AaABAg,@antoniomarcos2253,5/5/2025,2025-05-05T18:28:51Z,0
2,UgyB6fA2bStIyrlSzfZ4AaABAg,@GuadalupeMartinez-d1l,😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂,2025-05-03T15:40:59Z,0
3,UgxSyUq63h_Qd6J-zs54AaABAg,@AdrianV.k,Alguém escutando essa em 2025,2025-05-03T03:25:31Z,1
4,UgykjfHouNj5_e_LI7Z4AaABAg,@djmegatronn,nunca entendi a treta desse clipe kkk,2025-05-02T16:26:25Z,0


In [27]:
# Carregar modelo Detoxify (na CPU)
modelo = Detoxify("multilingual", device="cpu")

In [29]:
# Função auxiliar para limpar a saída do notebook/terminal
def limpar_terminal():
    # Para Jupyter/IPython, descomente:
    clear_output(wait=True)  
    # Para terminal puro, descomente:
    # os.system("cls" if os.name == "nt" else "clear")

In [30]:
# Classificar mensagens e exibir progresso
tqdm.pandas(desc="Classificando")  # habilita barra na aplicação

def classificar(texto):
    """Retorna dict com 7 rótulos convertidos p/ float nativo."""
    predicoes = modelo.predict(texto)
    return {rotulo: float(valor) for rotulo, valor in predicoes.items()}

# Aplica função à coluna 'message', com progresso
seriePredicoes = dfComentarios["message"].progress_apply(classificar)

# Converte para DataFrame e concatena aos comentários originais
dfPredicoes = pd.json_normalize(seriePredicoes)
dfFinal = pd.concat([dfComentarios, dfPredicoes], axis=1)

Classificando: 100%|██████████| 144695/144695 [1:26:45<00:00, 27.80it/s]


> Classificação do Oscar (xxx comentários) em um Ryzen 5 5600 com 16GB RAM: 00:14:17

> Classificação do Monark (3798 comentários) em um Ryzen 5 5600 com 16GB RAM: 00:02:49

> Classificação do NegoBorel (144695 comentários) em um Ryzen 5 5600 com 16GB RAM: 01:26:

In [31]:
# Salvar resultado em JSON 
dfFinal.to_json(arquivoDeSaida, orient="records", force_ascii=False, indent=2)